In [ ]:
import numpy as np
import pImpactR as impact
from copy import deepcopy as copy
import matplotlib.pyplot as plt
import sys
np.set_printoptions(formatter={'float': lambda x: format(x, '1.2E')})

In [ ]:
NTURN = [128,256,512,1024,1024*2]

nCore_y = 4
nCore_z = 4

NL_c = 0.01
NL_t = 0.4
NL_nu= 0.3
NL_L = 1.8
dTune = 0.01


alfx = np.tan(np.pi*NL_nu)
betx = NL_L/np.sin(2.0*np.pi*NL_nu)

pipe_radius = 0.02*np.sqrt(2.0)

# 1. Preparation

### 1.1 read InputFile and simplify lattice

In [ ]:
beam, lattice = impact.readInputFile('test_toymodel.in');
beam.nCore_y = nCore_y
beam.nCore_z = nCore_z
beam.distribution.distribution_type = 'ReadFile'
beam.current = 0.0

ke = beam.kinetic_energy
mass = beam.mass
freq = beam.frequency

In [ ]:
def normalize(data):
    data[:,1] = (data[:,0]*alfx/np.sqrt(betx) + data[:,1]*np.sqrt(betx))/NL_c
    data[:,3] = (data[:,2]*alfx/np.sqrt(betx) + data[:,3]*np.sqrt(betx))/NL_c
    data[:,0] = data[:,0]/(np.sqrt(betx)*NL_c)
    data[:,2] = data[:,2]/(np.sqrt(betx)*NL_c)
    
def unnormalize(data):
    data[:,1] = (-data[:,0]*alfx + data[:,1])*NL_c/np.sqrt(betx)
    data[:,3] = (-data[:,2]*alfx + data[:,3])*NL_c/np.sqrt(betx)
    data[:,0] = data[:,0]*np.sqrt(betx)*NL_c
    data[:,2] = data[:,2]*np.sqrt(betx)*NL_c

In [ ]:
for i in range(len(lattice)-1,0,-1):
    if lattice[i].type == 'write_raw_ptcl':
        lattice.pop(i)
lattice[1].n_sckick = 100
lattice[1].length = NL_L
lattice[1].strength_t = NL_t
lattice[1].transverse_scale_c = NL_c
lattice[1].tune_advance = NL_nu
lattice[1].pipe_radius = 0.02*np.sqrt(2)

lattice[2].tune_advance_x = dTune
lattice[2].tune_advance_y = dTune
lattice[2].nonlinear_insert_tuneAdvance = NL_nu
lattice[2].nonlinear_insert_length = NL_L

### 1.2. generate particles

In [ ]:
testP = 0.0
k=0
q_m = beam.multi_charge.q_m[0]
pTest = []
nx = 100
xmax = 0.98
for x in np.linspace(-xmax,xmax,nx):
    for y in np.linspace(-xmax,xmax,nx):
        if x**2+y**2<=xmax**2 :
            k+=1
            pTest.append([x,0.0,y,0.0,0.0,0.0,q_m,0.0,k]) 
pTest = np.array(pTest)
npt = len(pTest)
beam.n_particles = npt
print('initial npt=',npt)

In [ ]:
unnormalize(pTest)
impact.writeParticleData(pTest, ke, mass, freq)

In [ ]:
lattice[0].turns = NTURN[0]

TBT = impact.getElem('TBT')
TBT.file_id = NTURN[0]
TBT.pID_begin = 1
TBT.pID_end = npt
lattice.insert(1,TBT)

OUTPUT = impact.getElem('write_raw_ptcl')
OUTPUT.file_id = NTURN[0]
OUTPUT.turn = NTURN[0]
OUTPUT.format_id = 2
lattice.append(OUTPUT)

# FMA

### Run Forward 

In [ ]:
impact.writeInputFile(beam,lattice) 
impact.run(beam)

In [ ]:
for nturn in NTURN[:-1]:
    lattice[0].turns = nturn
    TBT.file_id = nturn*2
    OUTPUT.turn = nturn
    OUTPUT.file_id = nturn*2
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = nturn
    impact.writeInputFile(beam,lattice) 
    impact.run(beam)

### Load Data

In [ ]:
iTBT = {}
TBT = {}
X = {}
Y = {}
Npt = {}

for nturn in NTURN:
    print(nturn)
    iTBT[nturn],TBT[nturn] = impact.readTBT(nturn,ke,mass,freq)
    TBT[nturn] = TBT[nturn][:,:4,:]
    Npt[nturn] = len(iTBT[nturn])

In [ ]:
for i in range(len(NTURN)):
    nturn = NTURN[i]
    
    npt = len(iTBT[nturn])
    X[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    Y[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    
    iturn = 0
    for j in range(i+1):
        jturn = NTURN[j]
        
        itmp = np.in1d(iTBT[jturn],iTBT[nturn])
        tmp = TBT[jturn][:,:,itmp]
        X[nturn][iturn:jturn] = (tmp[:,0,:] - 1j*(betx*tmp[:,1,:]+alfx*tmp[:,0,:]))/(NL_c*np.sqrt(betx))
        Y[nturn][iturn:jturn] = (tmp[:,2,:] - 1j*(betx*tmp[:,3,:]+alfx*tmp[:,2,:]))/(NL_c*np.sqrt(betx))
        print(iturn,jturn,nturn,np.sum(itmp),npt)
        
        iturn = jturn

### Apply NAFF

In [ ]:
def getClosestDiff(ref,arr):
    # ref.shape = nSample
    nMode = len(arr)
    if ref>1.0:
        ref = ref - 1.0
    for i in range(nMode):
        if arr[i]>1.0:
            arr[i]=arr[i]-1.0
    diff = np.abs(ref-arr)
    for i in range(nMode):
        if (diff[i]>0.5):
            diff[i] = 1.0 - diff[i]
    return np.min(diff)

In [ ]:
iMax = {}
iMin = {}
tuneXbefore= {}
tuneYbefore= {}
Xdiff = {}
Ydiff = {}
Adiff = {}

for nturn in NTURN[1:]:
    npt = len(X[nturn][0,:])
    tuneXbefore[nturn] = np.zeros(npt)
    tuneYbefore[nturn] = np.zeros(npt)
    Xdiff[nturn] = np.zeros(npt)
    Ydiff[nturn] = np.zeros(npt)
    Adiff[nturn] = np.zeros(npt)
    for i in range(npt):
        signal = X[nturn][:int(nturn/2),i] -np.mean(X[nturn][:int(nturn/2),i])
        tuneXbefore[nturn][i],amp,dummy = impact.util.naff(1,signal,window_id=1)
        signal = X[nturn][int(nturn/2):,i] -np.mean(X[nturn][int(nturn/2):,i])
        tune,amp,dummy = impact.util.naff(3,signal,window_id=1)
        Xdiff[nturn][i] = getClosestDiff(tuneXbefore[nturn][i],tune)
        
        signal = Y[nturn][:int(nturn/2),i] -np.mean(Y[nturn][:int(nturn/2),i])
        tuneYbefore[nturn][i],amp,dummy = impact.util.naff(1,signal,window_id=1)
        signal = Y[nturn][int(nturn/2):,i] -np.mean(Y[nturn][int(nturn/2):,i])
        tune,amp,dummy = impact.util.naff(3,signal,window_id=1)
        Ydiff[nturn][i] = getClosestDiff(tuneYbefore[nturn][i],tune)
        
    Adiff[nturn] = np.sqrt(Xdiff[nturn]**2 +Ydiff[nturn]**2)
    Adiff[nturn] = np.log10(Adiff[nturn]+1.0e-6)
    Xdiff[nturn] = np.log10(Xdiff[nturn]+1.0e-6)
    Ydiff[nturn] = np.log10(Ydiff[nturn]+1.0e-6)
    iMax[nturn] = np.argmax(Adiff[nturn]),np.argmax(Xdiff[nturn]),np.argmax(Ydiff[nturn])
    iMin[nturn] = np.argmin(Adiff[nturn]),np.argmin(Xdiff[nturn]),np.argmin(Ydiff[nturn])

### Plot

In [ ]:
n=NTURN[0]
xmax = np.max([np.real(X[n][0,:]),np.real(Y[n][0,:])])
for n in NTURN[1:]:
    fig = plt.figure(figsize=[4,3.3])
    f1=plt.scatter(np.real(X[n][0,:]),np.real(Y[n][0,:]),c=Adiff[n],s=1.6,marker='s')
    plt.scatter([-1.0,1.0],[0.0,0.0],c='r',s=77,marker='x')
    plt.xlim(-xmax*1.1,xmax*1.1)
    plt.ylim(-xmax*1.1,xmax*1.1)
    plt.xlabel('$x_n$')
    plt.ylabel('$y_n$')
    plt.colorbar(f1)
    plt.savefig('FMA.DA.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.turn='+str(int(n/2))+'.png',
                dpi=180, bbox_inches = "tight")


# FB

### Run Backward

In [ ]:
lattice.pop()
latticeB = copy(lattice[1:][::-1])
latticeB.insert(0,copy(lattice[0]))

for i in range(len(latticeB)):
    if 'length' in latticeB[i]:
        latticeB[i].length = -latticeB[i].length
    if latticeB[i].type == 'linear_matrix_map':
        latticeB[i].tune_advance_x                 =-latticeB[i].tune_advance_x
        latticeB[i].tune_advance_y                 =-latticeB[i].tune_advance_y
        latticeB[i].nonlinear_insert_tuneAdvance =-latticeB[i].nonlinear_insert_tuneAdvance
        latticeB[i].nonlinear_insert_length      =-latticeB[i].nonlinear_insert_length

In [ ]:
for nturn in NTURN[:-1]:
    latticeB[0].turns = nturn
    latticeB[-1].file_id = 100000+nturn
    beam.distribution.distribution_type = 'ReadFile_binary'
    beam.distribution.file_id = nturn
    impact.writeInputFile(beam,latticeB) 
    impact.run(beam)

### Load Data

In [ ]:
for nturn in NTURN[:-1]:
    print(nturn)
    iTBT[-nturn],TBT[-nturn] = impact.readTBT(100000+nturn,ke,mass,freq)
    TBT[-nturn] = TBT[-nturn][:,:4,:]
    Npt[-nturn] = len(iTBT[-nturn])

In [ ]:
for i in range(len(NTURN)-1):
    nturn = NTURN[i]
    
    npt = len(iTBT[-nturn])
    X[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    Y[nturn]=np.zeros((nturn,npt),dtype=np.complex)
    tmp = TBT[-nturn][::-1,:4,:]
    X[-nturn]=(tmp[:,0,:] - 1j*(betx*tmp[:,1,:]+alfx*tmp[:,0,:]))/(NL_c*np.sqrt(betx))
    Y[-nturn]=(tmp[:,2,:] - 1j*(betx*tmp[:,3,:]+alfx*tmp[:,2,:]))/(NL_c*np.sqrt(betx))
    
    iturn = 0
    for j in range(i+1):
        jturn = NTURN[j]
        
        itmp = np.in1d(iTBT[jturn],iTBT[-nturn])
        tmp = TBT[jturn][:,:,itmp]
        X[nturn][iturn:jturn] = (tmp[:,0,:] - 1j*(betx*tmp[:,1,:]+alfx*tmp[:,0,:]))/(NL_c*np.sqrt(betx))
        Y[nturn][iturn:jturn] = (tmp[:,2,:] - 1j*(betx*tmp[:,3,:]+alfx*tmp[:,2,:]))/(NL_c*np.sqrt(betx))
        print(iturn,jturn,nturn,np.sum(itmp),npt)
        
        iturn = jturn

In [ ]:
for nturn in NTURN[:-1]:
    nturn = -nturn
    Xdiff[nturn] = np.abs(X[nturn]-X[-nturn])**2
    Ydiff[nturn] = np.abs(Y[nturn]-Y[-nturn])**2
    Adiff[nturn] = Xdiff[nturn] + Ydiff[nturn]
    Xdiff[nturn] = np.sqrt(-np.sum(Xdiff[nturn],0)/nturn)
    Ydiff[nturn] = np.sqrt(-np.sum(Ydiff[nturn],0)/nturn)
    Adiff[nturn] = np.sqrt(-np.sum(Adiff[nturn],0)/nturn)
    Xdiff[nturn] = np.log10(Xdiff[nturn])
    Ydiff[nturn] = np.log10(Ydiff[nturn])
    Adiff[nturn] = np.log10(Adiff[nturn])
    iMax[nturn] = np.argmax(Adiff[nturn]),np.argmax(Xdiff[nturn]),np.argmax(Ydiff[nturn])
    iMin[nturn] = np.argmin(Adiff[nturn]),np.argmin(Xdiff[nturn]),np.argmin(Ydiff[nturn])

In [ ]:
n=NTURN[0]
xmax = np.max([np.real(X[n][0,:]),np.real(Y[n][0,:])])
for n in NTURN[:-1]:
    plt.figure(figsize=[4,3.3])
    f1=plt.scatter(np.real(X[n][0,:]),np.real(Y[n][0,:]),c=Adiff[-n],s=1.6,marker='s')
    plt.scatter([-1.0,1.0],[0.0,0.0],c='r',s=77,marker='x')
    plt.xlim(-xmax*1.1,xmax*1.1)
    plt.ylim(-xmax*1.1,xmax*1.1)
    plt.xlabel('$x_n$')
    plt.ylabel('$y_n$')
    plt.colorbar(f1)
    plt.tight_layout()
    plt.savefig('FB.DA.nu='+str(NL_nu)+'.tau='+str(NL_t)+'.mu='+str(dTune)+'.turn='+str(n)+'.png',dpi=180)